# Dependency Reduction

## Setup

In [206]:
import csv
import pandas as pd
from pandas import DataFrame
import os
import json

NPM libraries available on GitHiub dependency changes per quarter from 2014 to 2020.

In [207]:
dependency_path = "../data/dependencies.json"

### Load Dataset

In [208]:
#warning: this file is large
dependency_data = json.load(open(dependency_path))

package_dependencies = {}
package_repo_map = {}

for repo_name in dependency_data:
    i=+1
    data = dependency_data[repo_name]
    package_name = data['package_name']
    package_repo_map[package_name] = repo_name
    dcq_data = data["dependencies_changes_quarter"]
    package_dependencies[package_name] = {}
    for q in dcq_data:
        q_data = dcq_data[q]
        dependents = q_data['dependent']
        package_dependencies[package_name][q] = dependents

# del dependency_data

### Generate Dataframe

Generate a dataframe containing number of dependencies per Q per package, and earliest and latest dependency count.

In [209]:
#first, get all quarters
q_array = []
for p in package_dependencies:
    for q in package_dependencies[p]:
        if q not in q_array:
            q_array.append(q)
q_array.sort()

#next, generate matrix for dataframe
offset = 4
columns = ['package', 'earliest', 'latest', 'max_d'] + q_array
data = []
for p in package_dependencies:
    p_entry = [p] + ([None] * (len(q_array) + offset-1))
    for q in package_dependencies[p]:
        id = q_array.index(q)
        p_entry[id+offset] = len(package_dependencies[p][q])
    max_d = None
    earliest = p_entry[offset]
    for q in p_entry[offset:]:
        if earliest == None:
            earliest = q
        if max_d == None:
            max_d = q
        elif q != None and max_d < q:
            max_d = q
    latest = p_entry[len(p_entry)-1]
    for q in p_entry[:offset:-1]:
        if latest == None:
            latest = q
    p_entry[1] = earliest
    p_entry[2] = latest
    p_entry[3] = max_d
    data.append(p_entry)

dependency_df = pd.DataFrame(data, columns=columns)

## How many packages?

In [210]:
print('Total packages:', len(dependency_df))

zero_df = dependency_df.query('latest == 0')
print("Packages with no dependencies:", len(zero_df))

reduce_to_zero_df = zero_df.query('max_d != 0 and max_d.notna()')
print("Packages with zero that reduced:", len(reduce_to_zero_df))

reduce_df = dependency_df.query('max_d != latest')
print("Packages that reduced dependencies at some point:", len(reduce_df))

#this one is iffy if the package is older than 2014
reduce_from_earliest_df = reduce_df.query('latest < earliest')
print("Packages that have less dependencies than they started with:", len(reduce_from_earliest_df))

#just to compare
reduce_from_earliest_or_same_df = reduce_df.query('latest <= earliest')
print("Packages that reduced, and have less or same dependencies than they started with:", len(reduce_from_earliest_or_same_df))


Total packages: 97427
Packages with no dependencies: 1038
Packages with zero that reduced: 68
Packages that reduced dependencies at some point: 18682
Packages that have less dependencies than they started with: 10333
Packages that reduced, and have less or same dependencies than they started with: 11399


## Example cases

In [216]:
reduce_to_zero_names = list(reduce_to_zero_df['package'])
print(json.dumps(reduce_to_zero_names, indent=2))

print(json.dumps(dependency_data[package_repo_map[reduce_to_zero_names[4]]], indent=2))

[
  "wx-jssdk",
  "exprestify",
  "6lowpan",
  "mongoose-intl-konzek",
  "modular-adal-angular",
  "roketin-react-native-keyboard",
  "eslint-config-brightspace",
  "eslint-config-buildium",
  "eslint-config-button-platform",
  "eslint-config-standard-deviation",
  "fecs-git-hooks",
  "bat-ria",
  "fuller-dst-file",
  "eslint-config-hope",
  "three-trackballcontrols-es6",
  "utility2",
  "sprite-webpack-plugin",
  "react-font-awesome",
  "list-parsing",
  "astros-css-sprite",
  "eslint-config-lens",
  "selector-lite",
  "react-render-debugger",
  "react-render-visualizer-decorator",
  "obliviate",
  "acorn-es7-plugin",
  "mithril-resolver",
  "@meanie/express-jsonwebtoken",
  "meanie-mongoose-only-id",
  "meanie-mongoose-to-json",
  "mmallahham-first",
  "eslint-config-mx",
  "network-diagnostics",
  "eslint-config-naturalatlas",
  "eslint-config-vue-impression",
  "nodebb-plugin-beep",
  "react-fileprompt",
  "meanio-upload",
  "morsecodify",
  "pimatic-datalogger",
  "multi-progress"